In [1]:
%matplotlib inline   

# load the libraries
import matplotlib.pyplot as plt # 2D plotting library
import numpy as np              # package for scientific computing  

import math              # package for mathematics (pi, arctan, sqrt, factorial ...)

import cmath 
from sympy import symbols, I, exp, diff, lambdify
import numpy as np


In [28]:
"All lengths given in units of the wavelength lambda"
l_1 = [0.6, 0.5, 0.45, 0.45, 0.45] #antenna lengths. 
l_2 =[0.001, 0.001, 0.001, 0.001, 0.001] #antenna radii
l_3 = [(0,0), (0.5,0), (1,0), (1.5,0), (2,0)] #antenna position

V = [0, 1, 0, 0, 0] #voltage in each antenna

def G(p, q, k, z, z_1, L_1, L_2, L_3):
   
    """

    Parameters
    ----------
    p, q: antenna indexes
    k : wavenumber
    z : 
    z_1 : z'

    Returns so called impedance kernel
    -------
    """

    #print(L_2)
    if p == q:
        d = L_2[p]
    else:
        d = math.sqrt((L_3[p][0] - L_3[q][0])**2+ (L_3[p][1] - L_3[q][1])**2)
    #print(d)
    R = math.sqrt((z - z_1)**2 + d**2)
    #print(R)
    return (cmath.exp(complex(0, -k*R)))/R

def second_derivative_G(p, q, k, z, z_1, L_1, L_2, L_3):
    if p == q:
        d = L_2[p]
    else:
        dx = L_3[p][0] - L_3[q][0]
        dy = L_3[p][1] - L_3[q][1]
        d = math.sqrt(dx**2 + dy**2)

    delta_z = z - z_1
    R = math.sqrt(delta_z**2 + d**2)

    if R == 0:
        return 0  # Avoid division by zero (optional: add singularity handling)

    exp_term = cmath.exp(complex(0, -k * R))
    term1 = (-1j * k) / R**3
    term2 = 3 * (1j * k * R + 1) / R**5
    term3 = -k**2 * delta_z**2 / R**3

    return (term1 + term2 + term3) * exp_term

def Z(p, q, k):
    """
    Compute the coefficient of entry (p, q) of the impedance matrix

    """
    global L_1, L_2, L_3
    wavelength = 2*math.pi/k
    L_1 = [l*wavelength for l in l_1]
    L_2 = [l*wavelength for l in l_2]
    L_3 = [(x*wavelength, y*wavelength) for (x, y) in l_3]
    h_p = L_1[p]/2
    h_q = L_1[q]/2
    N = 50     # Even Number
    Z = 0
    eta = 376.7 #sqrt(mu_0/epsilon_0)
    
    for n in range(-N//2, N//2+1):
        z = (n*h_p)/N
        dz = h_p/N
        for m in range(-N//2, N//2+1):
              z_1 = (m*h_q)/N
              dz_1 = h_q/N
              g = G(p, q, k, z, z_1, L_1, L_2, L_3)
              g_sec = second_derivative_G(p, q, k, z, z_1, L_1, L_2, L_3)
              sinp = math.sin(k*(h_p - abs(z))))/(math.sin(k*h_p)))*(math.sin(k*(h_q- abs(z_1)))/(math.sin(k*h_q)
              sinq = math.sin(k*(h_q- abs(z_1)))/(math.sin(k*h_q))
              Z += complex(0, eta/(4*math.pi*k))*sinp*sinq*(g*k**2+g_sec)*dz*dz_1
              
    return Z   

In [29]:
def matrixZ(k):
    n = len(l_1)
    matZ = np.matrix([[Z(p,q,k) for q in range(0,n)] for p in range(0,n)])
    return matZ

In [30]:
def input_current(V,Z):
    I = np.linalg.solve(Z, V)
    return I  

In [31]:
matrixZ(3)

matrix([[ 2.70466022e+07+5.98254409e+12j,
          1.13156953e+01-3.18210091e+01j,
          3.22609769e-02+1.13597677e+01j,
         -1.88594088e-01-7.53681333e+00j,
          1.30602463e-01+5.65913899e+00j],
        [ 1.13156953e+01-3.18210091e+01j,
          1.53960025e+07+3.38006582e+12j,
          7.90714555e+00-2.09245967e+01j,
         -4.05344989e-02+8.54148571e+00j,
         -1.12486693e-01-5.65540790e+00j],
        [ 3.22609769e-02+1.13597677e+01j,
          7.90714555e+00-2.09245967e+01j,
          1.16289276e+07+2.53710167e+12j,
          6.98806280e+00-1.81673468e+01j,
         -5.78724908e-02+7.39463716e+00j],
        [-1.88594088e-01-7.53681333e+00j,
         -4.05344989e-02+8.54148571e+00j,
          6.98806280e+00-1.81673468e+01j,
          1.16289276e+07+2.53710167e+12j,
          6.98806280e+00-1.81673468e+01j],
        [ 1.30602463e-01+5.65913899e+00j,
         -1.12486693e-01-5.65540790e+00j,
         -5.78724908e-02+7.39463716e+00j,
          6.98806280e+00-1.816

In [32]:
matrixZ(100)

matrix([[ 1.47948241e+09+6.64727140e+15j,
          2.24437367e+04-8.77679771e+03j,
         -1.30610778e+03+2.68669076e+02j,
          2.59218338e+02-4.20022292e+01j,
         -8.15510989e+01+1.38781422e+01j],
        [ 2.24437367e+04-8.77679771e+03j,
          8.50655063e+08+3.75562880e+15j,
          1.49843779e+04-5.59247889e+03j,
         -9.84643224e+02+1.95160884e+02j,
          1.94751953e+02-3.05491699e+01j],
        [-1.30610778e+03+2.68669076e+02j,
          1.49843779e+04-5.59247889e+03j,
          6.47089884e+08+2.81900194e+15j,
          1.30703852e+04-4.80842134e+03j,
         -8.53345806e+02+1.66541836e+02j],
        [ 2.59218338e+02-4.20022292e+01j,
         -9.84643224e+02+1.95160884e+02j,
          1.30703852e+04-4.80842134e+03j,
          6.47089884e+08+2.81900194e+15j,
          1.30703852e+04-4.80842134e+03j],
        [-8.15510989e+01+1.38781422e+01j,
          1.94751953e+02-3.05491699e+01j,
         -8.53345806e+02+1.66541836e+02j,
          1.30703852e+04-4.808

In [35]:
V = [0,1,0,0,0]
input_current(V,matrixZ(100))

array([ 8.99019620e-28-3.51568145e-28j,  6.03098391e-23-2.66266996e-16j,
        1.41533992e-27-5.28233314e-28j, -9.30038441e-29+1.84337520e-29j,
        1.83951704e-29-2.88549359e-30j])

In [34]:
((math.sin(k*(h_q- abs(z_1)))/(math.sin(k*h_q))))

NameError: name 'k' is not defined